In [54]:
from datetime import date, timedelta, datetime
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import requests
from newspaper import Article
from multiprocessing import Pool, cpu_count
from tqdm import tqdm_notebook as tqdm
import pickle
import pandas as pd

In [2]:
startDate = date(2016, 1, 1)
endDate = datetime.now().date() + timedelta(days=1)
offset = 36892
newsStartDate = date(2001, 1, 1)

In [3]:
def generateLinkForDate(dateGiven):
    def startTime(x): return (x - newsStartDate).days + offset
    stringDate = str(dateGiven).replace('-', '/')
    return "https://timesofindia.indiatimes.com/%s/archivelist/year-%d,month-%d,starttime-%d.cms" % (stringDate, dateGiven.year, dateGiven.month,  startTime(dateGiven))

In [4]:
'''
    Returns the links that have the given queryWords in
    their headlines be used for scrapping
'''
def fetchRelevantNewsLinks(tagSet, queryWords):
    links = []
    for tag in tagSet:
        occurs = False
        headline = tag.text
        link = tag.get('href')
        
        if headline == None or link == None:
            continue
        headlineLower = headline.lower()
        for query in queryWords:
            if query in headline:
                occurs = True
                break

        if occurs:
            # Fix for links that don't start with http
            if not link.startswith("http"):
                link = "http://timesofindia.indiatimes.com/" + link
            links.append(link)
    return links

In [46]:
def getArticle(url, beginsWithLocation = True):
    try:
        article = Article(url)
        article.download()
        article.parse()
        #Return the location too
        if beginsWithLocation:
            location = article.text[:article.text.find(':')].strip()
            return article.title, article.text, location
        return article.title, article.text
    except:
        return 0

In [47]:
queryWords = 'protest, riot, violence, violent, unrest, clash, bandh, rally, election, rule ,law, policy, demonstration, congress, INC, BJP, guv ,govt, government, political, politics, party, PM, CM, minister, polls'
queryWords = [word.strip().lower() for word in queryWords.split(',')]

In [48]:
# currDate = startDate
# newsLinks = {}
# while currDate != endDate:
#     print(currDate, end=' ')
#     link = generateLinkForDate(currDate)
#     request = requests.get(link, headers={'User-Agent': 'Mozilla/5.0'})
#     soup = BeautifulSoup(request.text, 'lxml')
#     tagSet = soup.find_all('a')
#     foundLinks = fetchRelevantNewsLinks(tagSet, queryWords)
#     print(len(foundLinks))
#     newsLinks[str(currDate)] = foundLinks
#     currDate += timedelta(days=1)

In [49]:
# len(newsLinks.keys())

In [50]:
newsLinks = pickle.load(open('TOI-dump.pkl', 'rb'))

In [55]:
full_news = []
for key in newsLinks.keys():
    param = newsLinks[key]
    print(key)
    with Pool(cpu_count() * 3) as p:
        full_news = list(tqdm(p.imap(getArticle, param), total=len(param)))
        try:
            full_news.remove(0)
        except:
            pass
    df = pd.DataFrame(data=full_news, columns=['title', 'text', 'location'])
    df.to_csv('TOI/%s.csv' % key)

2016-01-01



2016-01-02


KeyboardInterrupt: 

In [53]:
len(full_news)

78

In [14]:
getArticle(param[0])

('Manipur Congress accuses BJP of putting up insurgent-backed candidate',
 'IMPHAL: Congress\' Manipur unit has accused the BJP of putting up an insurgent-backed candidate for a Lok Sabha seat here and compromising national integrity in a bid to win the upcoming elections at any cost, a charge rejected by the latter as an attempt to malign the party.BJP\'s candidate H Shukhopao Mate for the Outer Manipur parliamentary constituency, which goes to polls on April 11, has been accused of being backed by insurgent groups in the state — Zomi Re-unification Organisation (ZRO) and Kuki National Organisation (KNO).The two outfits, which have signed "suspension of operation" agreement with Government of India (GOI) and are currently in peace talks, are reported to have written separate letters to BJP president Amit Shah, asking Mate be given the party\'s ticket.Mate is vice-president of the BJP Manipur unit and is incharge of the party\'s Scheduled Tribe Morcha "It is not just the Congress but e

In [16]:
full_news = full_news.remove(None)

In [17]:
full_news

In [ ]:
getArticle(newsLinks['2016-01-01'][0])